# Mietpreisprognose

Vorgehen
-aktuelle Daten Sammeln
-Mit hsytorischen Mietspiegeln zurückrechnen und somit hystorische Daten erstellen
-Atribute Finden, die ein vollkommenes Modell bilden könnten (hier Eingeschränkt, da Atribute/exogene VAriablen theoretisch immer weiter konkretisierbar)
-Gewichte der Atribute (zum Beispiel mittels Heat-Map)

Modell bauen:
-Prognose von Rafferenzmieten durch Schätzung mit Möglicher Anpassung von Atributen
-Schätzmodell Für Mieten, durch Standortnahe Refferenzmieten (Refferenzmieten können dabei Prognostiziert werden)



## Datenaufbereitung

### Hinzufügen von Koordinaten

Adressen mittels der Google-Geocode-API in Koordinaten umwandeln um nummerische Daten für weiteren Berechnung zu erhalten

In [ ]:
import json
import requests
import time
import os
from tqdm import tqdm

# Lade die Daten
def load_data(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        loaded_data = json.load(file)
    return loaded_data

# Sonderzeichen behandeln
def handle_special_characters(value):
    if isinstance(value, str):  # Überprüfen, ob der Wert ein String ist
        # Umlaute umwandeln
        value = value.replace("ß", "ss")
        value = value.replace("ä", "ae")
        value = value.replace("Ä", "Ae")
        value = value.replace("Ü", "Ue")
        value = value.replace("ü", "ue")
        value = value.replace("ö", "oe")
    return value

# Sonderzeichen in den geladenen Daten ersetzen
def handle_special_chars_in_data(loaded_data):
    decoded_data = []
    for dictionary in loaded_data:
        new_dict = {}
        for key, value in dictionary.items():
            new_dict[key] = handle_special_characters(value)
        decoded_data.append(new_dict)
    return decoded_data

# Funktion zum Formatieren der Adresse
def format_address(item):
    bundesland = item['bundesland']
    stadtteil = item['stadtteil']
    stadt = item['stadt']
    strasse = item.get('strasse', '')  # Falls 'strasse' nicht vorhanden ist, setze einen leeren String
    plz = item['plz']
    return f"{strasse}, {plz} {stadtteil}, {stadt}, {bundesland}"

# Gehe durch die ursprünglichen Daten und formatiere die Adressen
def format_addresses(decoded_data):
    new_data = []
    for item in tqdm(decoded_data, desc="Adressformatierung", total=len(decoded_data)):
        address = format_address(item)
        item['address'] = address
        new_data.append(item)
    return new_data

def geocode_google(address, api_key):
    # Geocodierung einer Adresse
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    if data['results']:
        location = data['results'][0]['geometry']['location']
        return location['lat'], location['lng']
    else:
        return None, None

# Speichern der endgültigen Daten in eine JSON-Datei
def save_to_json(data, output_filename):
    with open(output_filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

def main():
    api_key = "Private_Google_key!!!"
    input_filename = "trainingData.json"
    output_filename = "trainingData_located.json"
    
    # Lade die Daten ein
    loaded_data = load_data(input_filename)

    # Handle special characters
    decoded_data = handle_special_chars_in_data(loaded_data)
    
    # Format addresses
    new_data = format_addresses(decoded_data)

    for item in new_data:
        address = item['address']

        # Geocodierung der Adresse
        latitude, longitude = geocode_google(address, api_key)

        # Fügt die geocodierten Daten zu den vorhandenen Daten hinzu
        if latitude and longitude:
            item['Latitude'] = latitude
            item['Longitude'] = longitude
        
    # Speichern der endgültigen Daten in eine JSON-Datei
    save_to_json(new_data, output_filename)

if __name__ == "__main__":
    main()

### Anzeigen der Datenpunkte

In [ ]:
import folium
import json

# Lese die JSON-Datei ein
with open('trainingData_located20XX.json', 'r') as file:
    data = json.load(file)

# Erstelle eine Karte von Deutschland
map = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

# Iteriere über die Datenpunkte und füge Marker zur Karte hinzu
for item in data:
    latitude = float(item['Latitude'])
    longitude = float(item['Longitude'])
    bundesland = item['bundesland']
    stadtteil = item['stadtteil']
    stadt = item['stadt']
    strasse = item.get('strasse', '')
    plz = item['plz']
    address = f"{strasse}, {plz} {stadtteil}, {stadt}, {bundesland}"

    # Erstelle einen blauen Punkt mit einer Pixelgröße von 2
    folium.CircleMarker(location=[latitude, longitude], radius=0.5, color='blue', fill=True, fill_color='blue').add_to(map)

# Zeige die Karte an
map

### Hystorische Daten mittels lokalen Mietpreisspiegeln erstellen

### Atribute

## Mietpreisschätzung nach Standorte in der Nähe

### Random Forest

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib
import json

# Lade die Daten
with open("trainingData_located20XX.json", "r") as file:
    data = json.load(file)

# Umwandeln in DataFrame
df = pd.DataFrame(data)

# Behandeln Sie Ja/Nein-Werte in den Daten
bool_columns = ["hasBasement", "hasBalcony", "hasGarden", "hasElevator"]
for column in bool_columns:
    X[column].fillna(0.5, inplace=True)

# HausTyp behandeln
df["houseType"] = df["houseType"].astype("category").cat.codes

# Ziehen Sie X (Merkmale) und y (Ziel) aus den Daten
X = df.drop(columns=["rent", "_class", "address", "Latitude", "Longitude", "stadt", "stadtteil", "plz", "strasse", "bundesland"])
y = df["rent"]

# Finden Sie Zeilen mit NaN-Werten
nan_rows = X[X.isna().any(axis=1)]
print(nan_rows)

# Prüfen auf NaN-Werte und ersetzen mit Durchschnittswert der Spalte
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# Teilen Sie die Daten in Trainings- und Testsets auf
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Erstellen Sie den Regressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Trainieren Sie das Modell
regressor.fit(X_train, y_train)

# Testen Sie das Modell
y_pred = regressor.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

# Speichern Sie das Modell
joblib.dump(regressor, 'rent_model.pkl')


Empty DataFrame
Columns: [roomCount, propertyAge, livingSpace, hasBasement, hasBalcony, parkingLotCount, hasGarden, hasElevator, houseType]
Index: []
RMSE: 255.760592


['rent_model.pkl']

### Neuronales Netz zur Prognose (ohne Refferenzpunkte)

In [11]:
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tqdm.keras import TqdmCallback

# Lade die Daten
with open("trainingData_located20XX.json", "r") as file:
    data = json.load(file)

# Umwandeln in DataFrame
df = pd.DataFrame(data)

# Behandeln Sie Ja/Nein-Werte in den Daten
bool_columns = ["hasBasement", "hasBalcony", "hasGarden", "hasElevator"]
for column in bool_columns:
    df[column] = df[column].map({True: 1, False: 0, np.nan: 0.5})

# HausTyp behandeln
df["houseType"] = df["houseType"].astype("category").cat.codes

# Berechnen Sie den Mietpreis pro Quadratmeter
df["rent_per_sqm"] = df["rent"] / df["livingSpace"]

# Ziehen Sie X (Merkmale) und y (Ziel) aus den Daten
X = df.drop(columns=["rent", "_class", "address", "Latitude", "Longitude", "stadt", "stadtteil", "plz", "strasse", "bundesland", "rent_per_sqm"])
y = df["rent_per_sqm"]

# Prüfen auf NaN-Werte und ersetzen mit Durchschnittswert der Spalte
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# Standardisieren Sie die Daten
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Teilen Sie die Daten in Trainings- und Testsets auf
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Erstellen Sie das Modell
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

# Kompilieren Sie das Modell
model.compile(loss='mean_squared_error', optimizer='adam')

# Trainieren Sie das Modell und speichern Sie die History für die Plotting
history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0, callbacks=[TqdmCallback(verbose=1)])

# Testen Sie das Modell
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE per sqm: %f" % (rmse))

# Speichern Sie das Modell
model.save('rent_per_sqm_model_nn.h5')

# Plotting
plt.plot(history.history['loss'])
plt.title('Model Loss Progression During Training/Validation')
plt.ylabel('Training and Validation Losses')
plt.xlabel('Epoch Number')
plt.show()
"""


100%|██████████| 50/50 [08:23<00:00, 10.08s/epoch, loss=nan]


859/859 [==============================] - 1s 1ms/step


ValueError: Input contains infinity or a value too large for dtype('float64').

### KI Modell mit Reffernezpunkte

In [18]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import json
import joblib
import numpy as np
import matplotlib.pyplot as plt
from tqdm.keras import TqdmCallback
from geopy.distance import geodesic
import warnings
warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)

# Lade die Daten
with open("trainingData_located20XX.json", "r") as file:
    data = json.load(file)

# Umwandeln in DataFrame und erstellen eines GeoDataFrame
df = pd.DataFrame(data)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# Create spatial index on gdf
gdf_sindex = gdf.sindex

# Behandeln Sie Ja/Nein-Werte in den Daten
bool_columns = ["hasBasement", "hasBalcony", "hasGarden", "hasElevator"]
for column in bool_columns:
    df[column] = df[column].map({True: 1, False: 0, np.nan: 0.5})

# HausTyp behandeln
df["houseType"] = df["houseType"].astype("category").cat.codes

# Berechnen Sie den Mietpreis pro Quadratmeter
df["rent_per_sqm"] = df["rent"] / df["livingSpace"]

# Berechnen Sie Merkmale für nahegelegene Wohnungen
df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf = gpd.GeoDataFrame(df, geometry='geometry')

def find_nearby(row, gdf, radius=0.02):  # adjust radius to represent approximate kilometers
    potential_matches_index = list(gdf_sindex.intersection(row['geometry'].buffer(radius).bounds))
    potential_matches = gdf.iloc[potential_matches_index]
    precise_matches = potential_matches[potential_matches.distance(row['geometry']) <= radius]
    if len(precise_matches) < 5:
        precise_matches = gdf.iloc[gdf_sindex.nearest(row['geometry'].bounds, 5)]
    return precise_matches['rent_per_sqm'].describe()

df[['count_nearby', 'mean_rent_per_sqm_nearby', 'std_rent_per_sqm_nearby', 'min_rent_per_sqm_nearby', '25%', '50%', '75%', 'max_rent_per_sqm_nearby']] = df.apply(lambda row: find_nearby(row, gdf), axis=1)
df = df.drop(columns=['geometry', '25%', '50%', '75%'])

# Ziehen Sie X (Merkmale) und y (Ziel) aus den Daten
X = df.drop(columns=["rent", "_class", "address", "Latitude", "Longitude", "stadt", "stadtteil", "plz", "strasse", "bundesland", "rent_per_sqm"])
y = df["rent_per_sqm"]

# Prüfen auf NaN-Werte und ersetzen mit Durchschnittswert der Spalte
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# Standardisieren Sie die Daten
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Teilen Sie die Daten in Trainings- und Testsets auf
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Erstellen Sie # das Modell
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

# Kompilieren Sie das Modell
model.compile(loss='mean_squared_error', optimizer='adam')

# Trainieren Sie das Modell und speichern Sie die History für die Plotting
history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0, callbacks=[TqdmCallback(verbose=1)])

# Testen Sie das Modell
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE per sqm: %f" % (rmse))

# Speichern Sie das Modell
model.save('rent_per_sqm_model_nn.h5')

# Plotting
plt.plot(history.history['loss'])
plt.title('Model Loss Progression During Training/Validation')
plt.ylabel('Training and Validation Losses')
plt.xlabel('Epoch Number')
plt.show()

c:\Users\Joel\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\sindex.py:665: FutureWarning: sindex.nearest using the rtree backend was not previously documented and this behavior is deprecated in favor of matching the function signature provided by the pygeos backend (see PyGEOSSTRTreeIndex.nearest for details). This behavior will be updated in a future release.
  warnings.warn(
c:\Users\Joel\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\sindex.py:665: FutureWarning: sindex.nearest using the rtree backend was not previously documented and this behavior is deprecated in favor of matching the function signature provided by the pygeos backend (see PyGEOSSTRTreeIndex.nearest for details). This behavior will be updated in a future release.
  warnings.warn(
c:\Users\Joel\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\sindex.py:665: FutureWarning: sindex.nearest using the rtree backend was not previously documented and this behavior 

KeyboardInterrupt: 

### Prognosen nach Atributen

## Ein/Ausgabemaske

In [ ]:
from flask import Flask, request, jsonify
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from geopy.distance import geodesic

app = Flask(__name__)

# Laden Sie Ihre Daten hier
data = pd.read_json('data.json')

# Erstellen und Trainieren Sie Ihr Modell hier
model = LinearRegression()
model.fit(data.drop('rent', axis=1), data['rent'])

@app.route('/predict', methods=['POST'])
def predict():
    # Pflichteingaben
    address = request.json['address']
    living_space = float(request.json['livingSpace'])

    # optionale Eingaben
    optional_fields = request.json.get('optionalFields', {})

    # Errechnen der Koordinaten der Adresse
    coordinates = geocode_address(address)
    
    if coordinates:
        nearest = find_nearest_coordinates(coordinates)

        # Erstellen Sie den Eingabedatensatz für das Modell
        input_data = {'livingSpace': living_space}

        # Hier hinzufügen des optionalen Feldes und der "nearest"-Daten
        # ...

        prediction = model.predict(pd.DataFrame([input_data]))

        return jsonify({'prediction': prediction[0]})

    return jsonify({'error': 'Kann die Adresse nicht geokodieren.'})

if __name__ == "__main__":
    app.run(debug=True)
